In [ ]:
import os 

import pandas as pd
import numpy as np

"""
# import radial color histogram code, which can be found here:
# https://github.com/gmorinan/radialColorHistogram
from radialColorHistogram.color_density import ColorDensity
from radialColorHistogram.radial_split import RadialSplitter
"""

from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet as wn
from collections import defaultdict
import regex as re
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer

nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
nltk.download('stopwords')

try:
  import contractiond
except:
  !pip install contractions
  import contractions

tweet_tokenizer = TweetTokenizer()
# word_Lemmatized = WordNetLemmatizer()
stop = set(stopwords.words("english")) 
stemmer = SnowballStemmer(language="english")

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

try:
  from zeugma.embeddings import EmbeddingTransformer
except:
  !pip install zeugma
  from zeugma.embeddings import EmbeddingTransformer


from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

try:
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter
except:
  !pip install textattack
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 286kB 6.6MB/s 
     |████████████████████████████████| 327kB 9.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85397 sha256=48b94430d0e4f70f35c52e3d8cd387d11a91ea78a1524fd071309093fab81a07
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
  Created wheel for zeugma: filename=zeugma-0.48-cp37-none-any.whl size=8778 sha256=601fbee493e80d3f3f9552d65ddd368b6f3e08f0bd9d9231aa554b7fb5f0dda5
  Stored in directory: /root/.cache/pip/wheels/6a/b5/bc/5183ac478b0071d04d3ed0c0dd4a43db94c5c8ffb317b5eb53
Successfully built zeugma
     |████████████████████████████████| 358kB 7.6MB/s 
     |███████████████

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

### PP

In [ ]:
root = "/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/"
train_path = "train_images/train_images"
test_path = "test_images"

train_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train.csv")
test_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/test.csv")
train_dt.head()

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0
3,4,448_image_batch_2.png,"""COME ON MAN, YOU KNOW THE THING.\r\nJUST ASK ...",troll,2
4,5,653_image_batch_2.png,"""Those who believe without reason cannot be co...",none,0


In [ ]:
def preprocess_txt(text):

  caps = True if text != text.lower() else False
  multiline = True if '\n' in text else False
  bullet = True if '- ' in text else False

  text = text.lower()   # ??
  text = re.sub(r"\n"," ",text)
  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)
  
  # contractions fis
  expanded_words = []
  for word in text.split():
    expanded_words.append(contractions.fix(word))
  text = " ".join(expanded_words)

  
  #text = text.replace("@","@ ")
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text) # ????
  ## text = re.sub('@[^\s]+','',text)
  #####text = tweet_tokenizer.tokenize(text)
  
  # Single character removal
  text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
  # Removing multiple spaces
  text = re.sub(r'\s+', ' ', text)
  
  text = word_tokenize(text)

  text = [x for x in text if x not in stop]
  
  #text = [stemmer.stem(x) for x in text]

  ##text = ["<CAPS>"] + text if caps else text
  text = ["<MULTILINE>"] + text if multiline else text
  ##text = ["<BULLET>"] + text if bullet else text


  text = " ".join(text)


  return text


def preprocess_txt_mini(text):

  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)

  text = re.sub(r'[^a-zA-Z0-9 \.,\?\\\'\"\:\;\#\%\!\&\-\@\n]', ' ', text) # ????

  return text


def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(stopwords.words("english")) 

In [ ]:
def get_files_labels(subpath,csv):
    """Gets paths images within a folder, and their labels"""
    img_paths, img_labels, texts = [], [], []
    df = pd.read_csv(root+csv)
    i=0
    for index, row in df.iterrows():
        try:
          img, text, label = row["image id"], row["text"], int(row["label_num"])
        except:
          img, text, label = row["image id"], row["text"], -1
        # print(img_,": ",text_)
        
        ## Change this for img....>>>
        if not (i in [858, 1986, 1990] and csv=="train.csv"):
        #### if True:
          img_paths.append(root+subpath+'/'+img.strip())
          img_labels.append(label)
          ###texts.append(preprocess_txt(text))
          ###texts.append(text)
          texts.append(preprocess_txt_mini(text))
        i+=1
        #print('\r images: {} labels : {}'.format(len(img_paths), len(img_labels)) )
    return img_paths, img_labels, texts

In [ ]:
files_, y_, texts_ = get_files_labels(train_path,"train.csv")
files_test, y_test, texts_test = get_files_labels(test_path,"test.csv")


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    np.column_stack([files_, texts_]), y_, test_size=0.1, random_state=42, stratify=y_, shuffle=True)
X_test = np.column_stack([files_test,texts_test])

texts_[0]
X_train[0]

array(['/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train_images/train_images/image_6373.png',
       'When you finally drop that toxic girl in your life and you start glowing  The Internet Scavengers'],
      dtype='<U545')

In [ ]:
##aug = EmbeddingAugmenter()
##aug.augment(texts_[0])

In [ ]:
def newaug(sentences,aug = EmbeddingAugmenter(),easy=False ):
  new_sentences = []
  for sentence in sentences:
    if easy:
      sen = aug.augment(sentence)
      idx,le = 0,0
      for i in range(len(sen)):
        if len(sen[i]) > le:
          le = len(sen[i])
          idx = i
      new_sentences.append(sen[idx])     
    else:
      new_sentences.append(aug.augment(sentence)[0])
  return new_sentences

# newaug(X_train[:,1])[0]
newaug(["Hi from New York!","I'm fine!"],WordNetAugmenter())

['Hawaii from New York!', "I'm ok!"]

In [ ]:
#X_train_wn = newaug(X_train[:,1], aug=WordNetAugmenter())
#print("Done0")
X_train_emb = newaug(X_train[:,1], aug=EmbeddingAugmenter())
print("Done1")
#X_train_ed = newaug(X_train[:,1], aug=EasyDataAugmenter(), easy=True)


Done1


In [ ]:
## print(X_train[0,1])
for i in range(X_train.shape[0]):
  X_train[i,1] = preprocess_txt(X_train[i,1])

for i in range(X_val.shape[0]):
  X_val[i,1] = preprocess_txt(X_val[i,1])

for i in range(X_test.shape[0]):
  X_test[i,1] = preprocess_txt(X_test[i,1])

In [ ]:
for i in range(X_train.shape[0]):
  #X_train_wn[i] = preprocess_txt(X_train_wn[i])
  X_train_emb[i] = preprocess_txt(X_train_emb[i])
  #X_train_ed[i] = preprocess_txt(X_train_ed[i])


In [ ]:
import pickle 

open_file = open("X_train_emb1.pkl", "rb")
X_train_emb = pickle.load(open_file)
open_file.close()


open_file = open("X_train_wn.pkl", "rb")
X_train_wn = pickle.load(open_file)
open_file.close()


In [ ]:
def augment(sentences,n):
    import random
    random.seed(42)
    new_sentences = []
    for _ in range(n):
      for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        new_sentences.append(' '.join(words))
    return new_sentences

In [ ]:
def get_synonyms_lexicon(path="ppdb-xl.txt"):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path).readlines()]
    for e in text_entries:
        e = e.split(' ')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

synonyms = get_synonyms_lexicon()

In [ ]:
X_train2 = augment(X_train[:,1],1)
X_train[:,1][0],X_train2[0]

('finally drop toxic girl life start glowing internet scavengers',
 'girl glowing internet life scavengers toxic start finally drop')

In [ ]:
X_train1 = []
for sentence in X_train[:,1]:
  new_sentence = []
  for word in sentence.split():
    if word in synonyms.keys():
      new_sentence.append(synonyms[word][0])
    else:
      new_sentence.append(word)
  X_train1.append(" ".join(new_sentence))

X_train1[0]

'finally decline toxic girl living start glowing web scavengers'

# New Section

In [ ]:
#! unzip /content/drive/MyDrive/colab/data/glove.6B.zip

Archive:  /content/drive/MyDrive/colab/data/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
#X_train__ = X_train.copy()
#X_val__ = X_val.copy()

In [ ]:
X_train = list(X_train__[:,1]) + X_train1 + X_train_wn  + X_train_emb
X_val = list(X_val__[:,1])

In [ ]:
X_train[0]

'finally drop toxic girl life start glowing internet scavengers'

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 48

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.50d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
from metrics import precision, recall, f1

model = Sequential()
embedding_layer = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(16))
#model.add(Dense(48,activation='relu'))
model.add(Dropout(.5))

model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1])

In [ ]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 48, 50)            388550    
_________________________________________________________________
lstm_21 (LSTM)               (None, 16)                4288      
_________________________________________________________________
dropout_21 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 51        
Total params: 392,889
Trainable params: 4,339
Non-trainable params: 388,550
_________________________________________________________________


In [ ]:
history = model.fit(np.array(X_train),np.array( 4*[[x==0,x==1,x==2] for x in y_train]), batch_size=16, epochs=64, verbose=1, 
                    validation_data=(np.array(X_val),np.array([[x==0,x==1,x==2] for x in y_val])) )



In [ ]:
y_val_pred_ = model.predict(X_val)

In [ ]:
y_val_pred = np.argmax(y_val_pred_,axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report([x for x in y_val], y_val_pred))

              precision    recall  f1-score   support

           0       0.60      0.10      0.17        60
           1       0.51      0.48      0.49        69
           2       0.39      0.69      0.49        70

    accuracy                           0.44       199
   macro avg       0.50      0.42      0.39       199
weighted avg       0.49      0.44      0.40       199



In [ ]:
model.save("lstm_len48_4_f40_2.h5")

In [ ]:

import pickle
"""
open_file = open("X_train_emb.pkl", "wb")
pickle.dump(X_train_emb, open_file)
open_file.close()
"""

open_file = open("X_train_emb.pkl", "rb")
X_train_emb = pickle.load(open_file)
open_file.close()

#loaded_list==X_train_emb

In [ ]:
open_file = open("lstm__X_test.pkl", "wb")
pickle.dump(X_test, open_file)
open_file.close()


In [ ]:
X_train[3]

array([2297,   69,  866, 2298, 1291,   14,  825,  577, 2299,  410, 2300,
       1770, 4900,   45,  101, 2301,  101, 2302, 2303,  118,  867,  242,
        868,  242, 2304,  101, 2305, 2306, 2307, 2308,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [ ]:
import pickle 
open_file = open("lstm__X_train_wn.pkl", "wb")
pickle.dump(X_train_wn, open_file)
open_file.close()

open_file = open("lstm__X_train_emb.pkl", "wb")
pickle.dump(X_train_emb, open_file)
open_file.close()

In [ ]:
X_val[0], 'sep', X_test[0]

(array([ 113,  228, 1017, 2033, 1624, 4534,    8, 1580, 3612,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], dtype=int32),
 'sep',
 array([  16,   41, 1386, 4577, 1686,   89,   97,  262, 4577,  441,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], dtype=int32))

In [ ]:
X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
X_test.shape

AttributeError: ignored

In [ ]:
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)

In [ ]:
from keras.models import load_model

mod = load_model("/content/lstm_len48_4_f40_2.h5",custom_objects=
                 {'precision':precision,
                  'recall':recall,
                  'f1':f1})
y_val_pred_ = mod.predict(X_val)
y_val_pred = np.argmax(y_val_pred_,axis=1)
from sklearn.metrics import classification_report
print(classification_report([x for x in y_val], y_val_pred))


              precision    recall  f1-score   support

           0       0.60      0.10      0.17        60
           1       0.51      0.48      0.49        69
           2       0.39      0.69      0.49        70

    accuracy                           0.44       199
   macro avg       0.50      0.42      0.39       199
weighted avg       0.49      0.44      0.40       199



In [ ]:
open_file = open("lstm__val_pred_2.pkl", "wb")
pickle.dump(y_val_pred_, open_file)
open_file.close()


In [ ]:
y_test_pred_ = mod.predict(X_test)

open_file = open("lstm__test_pred_2.pkl", "wb")
pickle.dump(y_test_pred_, open_file)
open_file.close()
